In [1]:
'''
    20251210/20251211:
        使用vLLM测试模型的mmlu指标--- Failed
        generate ckpt& save

'''

'\n    20251210/20251211:\n        使用vLLM测试模型的mmlu指标--- Failed\n        generate ckpt& save\n\n'

In [2]:
import transformers

import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

import util
import importlib

importlib.reload(util)      # 只能 reload 模块本身
from util import calc_ppl, get_test_data, get_open_ended_answer, get_open_ended_answer_vllm   # reload 后再重新 import 函数

import copy
import datasets
import copy

# try 2

In [3]:
'''
生成checkpoint

'''

'\n生成checkpoint\n\n'

In [4]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='auto'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map,weights_only=False ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

def set_neuron_zero(model, neuron_list):
    '''
    将指定的neuron设置为zero

    '''
    state_dict = model.state_dict()

    for ikey,_ in neuron_list:
        w_name = ikey.split('_index_')[0]
        neuron_index = int(ikey.split('_index_')[1])

        #set zero
        state_dict[w_name][neuron_index,:] =0 #1 #0 #-1  #-63.75#0
    return model
    

def save_model(model, tokenizer, save_dir):
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    

In [5]:
'''
kus mask neuron

'''
'''
    convert LRP_based neuron to LAPE format

'''
import re

def convert_LAPE_format(neuron_en, config):

    layer_list = [{'up_proj':[], 'gate_proj':[], 'down_proj':[]} for _ in range(config.num_hidden_layers)]

    if isinstance(neuron_en, dict):
        new_tmp = []
        for ineuron, iscore in neuron_en.items():
            new_tmp.append((ineuron, iscore))

        neuron_en = new_tmp
        
    
    for ineuron, iscore in neuron_en:
    
        
        m = re.search(r'layers?\.(\d+).mlp.(.+).weight_index_([0-9]+)', ineuron)
        
        layer_index = int(m.group(1))
        layer_type = m.group(2)
        layer_neuron_index = int(m.group(3))
        assert layer_type in ['up_proj', 'gate_proj', 'down_proj']
    
        layer_list[layer_index][layer_type].append(int(layer_neuron_index))

    return layer_list

def get_mask_neuron_model_LRP(model, activation_mask_dict, is_llama =True):

        # get state_dict 
        state_dict = model.state_dict()
        if activation_mask_dict:
            def factory(mask):
                def llama_forward_lrp(self, x):
                    '''
                        mask: {'up_proj':[...], 'gate_proj':[...], 'down_proj':[...]}
                        
                    '''
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    if 'gate_proj' in mask:
                        activation.index_fill_(2, mask['gate_proj'], 0)

                    if 'up_proj' in mask:
                        x = activation * gate_up[:, :, i // 2 :].index_fill_(2, mask['up_proj'], 0)
                    else:
                        x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    if 'down_proj' in mask:
                        x.index_fill_(2, mask['down_proj'], 0)
                    return x
                def llama_forward(self, x):
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    activation.index_fill_(2, mask, 0)
                    x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    return x
                def llama_forward_split(self, x):
                    gate_ = self.gate_proj(x)  # b, l, 2i
                    i = gate_.size(-1)
                    activation = F.silu(gate_)
                    #activation.index_fill_(2, mask, 0)

                    # test
                    print(activation.shape)
                    activation.index_fill_(2, torch.tensor(list(range(activation.shape[1]))), 0 )
                    x = activation * self.up_proj(x)
                    x = self.down_proj(x)
                    return x
    
                def bloom_forward(self, x: torch.Tensor):
                    x, _ = self.dense_h_to_4h(x)
                    x = self.gelu_impl(x)
                    x.index_fill_(2, mask, 0)
                    x, _ = self.dense_4h_to_h(x)
                    return x
    
                if is_llama:
                    return llama_forward_lrp
                else:
                    return bloom_forward
    
            for i, ilayer_mask_dict in enumerate(activation_mask_dict): 
                #print('ilayer:',i, layer_mask)
                #if is_llama:
                    # latest
                    #obj = model.llm_engine.model_executor.driver_worker.model_runner.model.model.layers[i].mlp
                #else:
                    # latest
                    #obj = model.llm_engine.model_executor.driver_worker.model_runner.model.transformer.h[i].mlp

                if is_llama:
                    #obj = model.llm_engine.driver_worker.model_runner.model.model.layers[i].mlp

                    ilayer_mask_dict_cuda = {}
                    for ikey ,ival in ilayer_mask_dict.items():
                        if len(ival)>0:
                            #ilayer_mask_dict_cuda[ikey] = torch.LongTensor(ival).to('cuda')
                            
                            state_dict[f'model.layers.{i}.mlp.{ikey}.weight'][torch.LongTensor(ival),:] = 0       
                else:
                    #obj = model.llm_engine.driver_worker.model_runner.model.transformer.h[i].mlp
                    assert 1 ==0
                
                
                #obj.forward = MethodType(factory(ilayer_mask_dict_cuda), obj)
        return model

In [6]:
'''
LAPE mask neuron

'''

def get_mask_neuron_model(model, activation_mask_path, need_lang, is_llama= True):

    # get state_dict 
    state_dict = model.state_dict()
    
    if activation_mask_path:
        activation_masks = torch.load(activation_mask_path)
    else:
        activation_masks = [None]
    
    final_output = []
    if is_llama:
        languages = ["en", "zh", "fr", "es", "vi", "id", "ja"]
    else:
        languages = ["en", "zh", "fr", "es", "vi", "id"]
    
    need_lang = [need_lang]#['en', 'zh', 'vi']
    
    for activation_mask, mask_lang in zip(activation_masks, languages):
    
        if mask_lang not in need_lang:continue
            
        print(f'get mask =====lang:{mask_lang}=====')
    
        
        if activation_mask:
            def factory(mask):
                def llama_forward(self, x):
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    activation.index_fill_(2, mask, 0)
                    x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    return x
                def llama_forward_split(self, x):
                    gate_ = self.gate_proj(x)  # b, l, 2i
                    i = gate_.size(-1)
                    activation = F.silu(gate_)
                    #activation.index_fill_(2, mask, 0)

                    # test
                    print(activation.shape)
                    activation.index_fill_(2, torch.tensor(list(range(activation.shape[1]))), 0 )
                    x = activation * self.up_proj(x)
                    x = self.down_proj(x)
                    return x
    
                def bloom_forward(self, x: torch.Tensor):
                    x, _ = self.dense_h_to_4h(x)
                    x = self.gelu_impl(x)
                    x.index_fill_(2, mask, 0)
                    x, _ = self.dense_4h_to_h(x)
                    return x
    
                if is_llama:
                    return llama_forward
                else:
                    return bloom_forward
    
            for i, layer_mask in enumerate(activation_mask): 
                #print('ilayer:',i, layer_mask)
                if is_llama:
                    #obj = model.llm_engine.driver_worker.model_runner.model.model.layers[i].mlp
                    
                    # just mask neuron
                    # only masj gate proj
                    
                    #model.model.layers[i].mlp.gate_proj = 
                    state_dict[f'model.layers.{i}.mlp.gate_proj.weight'][layer_mask,:] = 0
                    
                    
                else:
                    #obj = model.llm_engine.driver_worker.model_runner.model.transformer.h[i].mlp
                    assert 1==0
                #obj.forward = MethodType(factory(layer_mask.to('cuda')), obj)
    return model

In [7]:
org_model_path ='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-hf'

In [8]:
model, tokenizer = load_model(org_model_path)

checkpoint: /root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
config = AutoConfig.from_pretrained(org_model_path)

# Generate LAPE MODEL

In [9]:
'''
get LAPE model

'''
import gc
save_dir='/root/autodl-tmp/llama2_7b_base_BaseNeuron_BaseMask_lang_{}'

lang_list = ['zh']#['en', 'vi', 'zh']
activation_mask_path='/root/autodl-fs/Language-Specific-Neurons/LLaMA-2-7B.neuron.pth'
for ilang in lang_list:
    
    tmp_model = get_mask_neuron_model(copy.deepcopy(model), activation_mask_path, ilang)
    save_model(model, tokenizer, save_dir.format(ilang))

    tmp_model.cpu()
    del tmp_model
    gc.collect()



get mask =====lang:zh=====


# generate LRP MODEL

In [10]:
model.state_dict()[f'model.layers.0.mlp.gate_proj.weight'].sum()

tensor(-684., device='cuda:0', dtype=torch.bfloat16)

In [ ]:
'''
get lrp model

'''
import gc
save_dir='/root/autodl-tmp/LRP_llama2_7b_base_BaseNeuron_BaseMask_lang_{}'

lang_list = ['en', 'vi', 'zh']
activation_mask_path_map = {
    'en': '/root/autodl-fs/LRP_kur_res/20251210_5000samples_cal_llama2_7b_base/th_1_selected_LRP_kur_res_en_zscore.pt',
    'vi': '/root/autodl-fs/LRP_kur_res/20251210_5000samples_cal_llama2_7b_base/th_1_selected_LRP_kur_res_vi_zscore.pt',
    'zh': '/root/autodl-fs/LRP_kur_res/20251210_5000samples_cal_llama2_7b_base/th_1_selected_LRP_kur_res_zh_zscore.pt'
}
for ilang in lang_list:
    # LRP version
    tmp_model = get_mask_neuron_model_LRP(copy.deepcopy(model), convert_LAPE_format(torch.load(activation_mask_path_map[ilang]), config), ilang)
    save_model(model, tokenizer, save_dir.format(ilang))

    tmp_model.cpu()
    del tmp_model
    gc.collect()



In [ ]:
!ls /root/autodl-fs/LRP_kur_res/20251210_5000samples_cal_llama2_7b_base/th_1_selected_LRP_kur_res_en_zscore.pt

In [11]:
tmp_model.state_dict()[f'model.layers.0.mlp.gate_proj.weight'].sum()

tensor(-668., device='cuda:0', dtype=torch.bfloat16)

In [12]:
model.state_dict()[f'model.layers.0.mlp.gate_proj.weight'].sum()

tensor(-684., device='cuda:0', dtype=torch.bfloat16)

In [11]:
tmp_model.state_dict()[f'model.layers.0.mlp.gate_proj.weight'].sum()

tensor(-720., device='cuda:0', dtype=torch.bfloat16)

In [9]:
#model.state_dict().keys()

In [11]:
model.model.layers[0]

LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
)

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [6]:
import os
save_name = 'llama2_7b_base_LAPE_mask_'
save_path = '/root/autodl-tmp/'
save_path = os.path.join(save_path, save_name)


# LAPE
neuron_path = ''

# try 1

In [3]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='cuda'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

In [4]:




def get_mask_neuron_model(model, activation_mask_path, need_lang):

    if activation_mask_path:
        activation_masks = torch.load(activation_mask_path)
    else:
        activation_masks = [None]
    
    final_output = []
    if is_llama:
        languages = ["en", "zh", "fr", "es", "vi", "id", "ja"]
    else:
        languages = ["en", "zh", "fr", "es", "vi", "id"]
    
    need_lang = [need_lang]#['en', 'zh', 'vi']
    
    for activation_mask, mask_lang in zip(activation_masks, languages):
    
        if mask_lang not in need_lang:continue
            
        print(f'get mask =====lang:{mask_lang}=====')
    
        
        if activation_mask:
            def factory(mask):
                def llama_forward_org(self, x):
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    activation.index_fill_(2, mask, 0)
                    x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    return x
                def llama_forward(self, x):
                    gate_ = self.gate_proj(x)  # b, l, 2i
                    i = gate_.size(-1)
                    activation = F.silu(gate_)
                    activation.index_fill_(2, mask, 0)
                    x = activation * self.up_proj(x)
                    x = self.down_proj(x)
                    return x
    
                def bloom_forward(self, x: torch.Tensor):
                    x, _ = self.dense_h_to_4h(x)
                    x = self.gelu_impl(x)
                    x.index_fill_(2, mask, 0)
                    x, _ = self.dense_4h_to_h(x)
                    return x
    
                if is_llama:
                    return llama_forward
                else:
                    return bloom_forward
    
            for i, layer_mask in enumerate(activation_mask):
                if is_llama:
                    obj = model.model.layers[i].mlp
                else:
                    obj = model.model.layers[i].mlp
                obj.forward = MethodType(factory(layer_mask.to('cuda')), obj)
    return model








def get_mask_neuron_model_vllm(model, activation_mask_path, need_lang):

    if activation_mask_path:
        activation_masks = torch.load(activation_mask_path)
    else:
        activation_masks = [None]
    
    final_output = []
    if is_llama:
        languages = ["en", "zh", "fr", "es", "vi", "id", "ja"]
    else:
        languages = ["en", "zh", "fr", "es", "vi", "id"]
    
    need_lang = [need_lang]#['en', 'zh', 'vi']
    
    for activation_mask, mask_lang in zip(activation_masks, languages):
    
        if mask_lang not in need_lang:continue
            
        print(f'get mask =====lang:{mask_lang}=====')
    
        
        if activation_mask:
            def factory(mask):
                def llama_forward(self, x):
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    activation.index_fill_(2, mask, 0)
                    x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    return x
                def llama_forward_split(self, x):
                    gate_ = self.gate_proj(x)  # b, l, 2i
                    i = gate_.size(-1)
                    activation = F.silu(gate_)
                    #activation.index_fill_(2, mask, 0)

                    # test
                    print(activation.shape)
                    activation.index_fill_(2, torch.tensor(list(range(activation.shape[1]))), 0 )
                    x = activation * self.up_proj(x)
                    x = self.down_proj(x)
                    return x
    
                def bloom_forward(self, x: torch.Tensor):
                    x, _ = self.dense_h_to_4h(x)
                    x = self.gelu_impl(x)
                    x.index_fill_(2, mask, 0)
                    x, _ = self.dense_4h_to_h(x)
                    return x
    
                if is_llama:
                    return llama_forward
                else:
                    return bloom_forward
    
            for i, layer_mask in enumerate(activation_mask): 
                #print('ilayer:',i, layer_mask)
                #if is_llama:
                    # latest
                    #obj = model.llm_engine.model_executor.driver_worker.model_runner.model.model.layers[i].mlp
                #else:
                    # latest
                    #obj = model.llm_engine.model_executor.driver_worker.model_runner.model.transformer.h[i].mlp

                if is_llama:
                    obj = model.llm_engine.driver_worker.model_runner.model.model.layers[i].mlp
                else:
                    obj = model.llm_engine.driver_worker.model_runner.model.transformer.h[i].mlp
                obj.forward = MethodType(factory(layer_mask.to('cuda')), obj)
    return model

In [5]:
import os
os.environ["VLLM_ENABLE_V1_MULTIPROCESSING"] = "0"

In [6]:
import argparse
from types import MethodType

import numpy as np
import torch
import torch.nn.functional as F
from vllm import LLM, SamplingParams
from tqdm import tqdm



#model_path ='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf'

model_path ='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-hf'
activation_mask_path='/root/autodl-fs/Language-Specific-Neurons/LLaMA-2-7B.neuron.pth'

is_llama = bool(model_path.find('llama') >= 0)
#model, tokenizer = load_model(model_path)

# 初始化 vLLM
llm =LLM(model=model_path, tensor_parallel_size=torch.cuda.device_count(), enforce_eager=True) #LLM(model=model_path, tensor_parallel_size=1)




INFO 12-10 15:18:02 llm_engine.py:70] Initializing an LLM engine with config: model='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-hf', tokenizer='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=True, seed=0)
INFO 12-10 15:21:44 llm_engine.py:275] # GPU blocks: 1923, # CPU blocks: 512


In [24]:
import re
import random
from tqdm import tqdm
from collections import defaultdict
from datasets import load_dataset
from vllm import SamplingParams


LETTER_MAP = ["A", "B", "C", "D"]


def extract_answer(text: str):
    # Parse A/B/C/D
    m = re.search(r'\b([ABCD])\b', text.upper())
    return m.group(1) if m else None


def build_5shot_prompt(subject, train_samples, test_item):
    # choose 5 examples
    shots = random.sample(train_samples, 5)

    prompt = f"The following are multiple choice questions (with answers) about {subject}.\n\n"
    
    for item in shots:
        prompt += "Q: " + item["question"] + "\n"
        for i, ch in enumerate(item["choices"]):
            prompt += f"{LETTER_MAP[i]}. {ch}\n"
        prompt += f"Answer: {LETTER_MAP[item['answer']]}\n\n"

    # test question
    prompt += "Q: " + test_item["question"] + "\n"
    for i, ch in enumerate(test_item["choices"]):
        prompt += f"{LETTER_MAP[i]}. {ch}\n"
    prompt += "Answer:"
    return prompt


def eval_mmlu_vllm_5shot(llm, max_new_tokens=4, verbose=True):
    ds = load_dataset("cais/mmlu", 'all')
    train = ds["train"]
    test = ds["test"]

    # group by subject
    train_groups = defaultdict(list)
    test_groups = defaultdict(list)
    
    for item in train:
        train_groups[item["subject"]].append(item)
    for item in test:
        test_groups[item["subject"]].append(item)

    sampling_params = SamplingParams(
        max_tokens=max_new_tokens,
        temperature=0.0
    )

    subject_scores = {}
    total_correct = 0
    total_count = 0

    subjects = sorted(train_groups.keys())
    if verbose:
        pbar = tqdm(subjects, desc="Evaluating MMLU 5-shot")

    for subject in subjects:
        train_samples = train_groups[subject]
        test_samples = test_groups[subject]

        prompts = []
        gt = []

        # build prompts
        for item in test_samples:
            prompts.append(build_5shot_prompt(subject, train_samples, item))
            gt.append(LETTER_MAP[item["answer"]])

        # batch inference
        outputs = llm.generate(prompts, sampling_params)

        correct = 0
        for out, label in zip(outputs, gt):
            text = out.outputs[0].text.strip()
            pred = extract_answer(text)
            if pred == label:
                correct += 1

        acc = correct / len(gt)
        subject_scores[subject] = acc

        total_correct += correct
        total_count += len(gt)

        if verbose:
            pbar.set_postfix({subject: f"{acc:.3f}"})
            pbar.update(1)

    overall = total_correct / total_count
    return overall, subject_scores


In [25]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [19]:
ds = load_dataset("cais/mmlu", 'all')

all/test-00000-of-00001.parquet:   0%|          | 0.00/3.50M [00:00<?, ?B/s]

all/validation-00000-of-00001.parquet:   0%|          | 0.00/408k [00:00<?, ?B/s]

all/dev-00000-of-00001.parquet:   0%|          | 0.00/76.5k [00:00<?, ?B/s]

all/auxiliary_train-00000-of-00001.parqu(…):   0%|          | 0.00/47.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

In [26]:
overall_acc, scores = eval_mmlu_vllm_5shot(llm)
print("Overall MMLU:", overall_acc)
print(scores)


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/cais/mmlu/resolve/main/README.md (Caused by ProxyError('Cannot connect to proxy.', TimeoutError('_ssl.c:980: The handshake operation timed out')))"), '(Request ID: bbc938b9-de82-4260-944b-8ed2be8edc9c)')' thrown while requesting HEAD https://huggingface.co/datasets/cais/mmlu/resolve/main/README.md
Retrying in 1s [Retry 1/5].


KeyError: 'train'